<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/llama_2_13b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip -q install bitsandbytes accelerate transformers

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline, BitsAndBytesConfig


In [3]:
model_id = "chenshake/Llama-2-7b-chat-hf-guanaco"

In [4]:
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config,
                                             device_map="auto")

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

CPU times: user 22.6 s, sys: 21 s, total: 43.6 s
Wall time: 4min 24s


In [7]:
%%time
# Run text generation pipeline with our next model
prompt = "Tell me about AI"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Tell me about AI [/INST] Artificial intelligence (AI) is intelligence demonstrated by machines, such as computers, robots, and software.его capabilities are based on the ability to learn, reason, and adapt to new situations, just like humans. AI is a rapidly growing field, with many applications in areas such as healthcare, finance, transportation, and entertainment.

There are several types of AI, including:

1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as playing chess or recognizing faces.
2. General or strong AI: This type of AI is designed to perform any intellectual task that a human can, such as reasoning, problem-solving, and learning.
3. Superintelligence: This type of AI is significantly more intelligent than the best human minds, and is capable of solving complex problems that
CPU times: user 14.7 s, sys: 369 ms, total: 15.1 s
Wall time: 15.9 s


In [8]:
prompt = "Explain a large language model and its architecture?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Explain a large language model and its architecture? [/INST] A large language model is a type of artificial intelligence that is trained on a large dataset of text. Unlike smaller language models, which are trained on a single dataset or a small number of datasets, large language models are trained on a large number of datasets, often numbering in the hundreds or thousands. This allows the model to learn a much wider range of language, including more complex and nuanced expressions.

The architecture of a large language model is typically composed of several layers of neural networks, each of which is trained on a different subset of the data. The first layer is typically a word embeddings layer, which maps each word in the dataset to a dense vector in a high-dimensional space. This allows the model to capture the meaning of each word in the dataset.

The next layer is typically a recurrent neural network (RNN) layer, which processes the sequence of words


In [9]:
%%time
prompt = "Tell me about Facebook boss"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Tell me about Facebook boss [/INST] Mark Zuckerberg is the CEO of Meta, the company that owns Facebook. He is a billionaire and a well-known figure in the tech industry. 
CPU times: user 3.05 s, sys: 127 ms, total: 3.18 s
Wall time: 3.17 s
